This notebook is used to get predictions from a customized model ( trained in local then I push the docker image)

I followed the following steps: [Containerize and run training code locally](https://cloud.google.com/vertex-ai/docs/training/containerize-run-code-local?hl=fr)

In [12]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("vertex-ai-tuto-380714-811aa46f0cfa.json")

# Reading data 

In [1]:
import pandas as pd 

In [6]:
df = pd.read_csv('data/iris.csv')
df = df.iloc[:, :-1]
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,8748.0,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.195556,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.0
1,27074.0,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.177675,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.0
2,28292.0,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,-0.170425,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.0
3,28488.0,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,-0.221677,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.0
4,31392.0,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,1.177573,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.0


# Create an endpoint 

[Get predictions from a custom trained model](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions?hl=fr)

In [13]:
from google.cloud import aiplatform

def create_endpoint_sample(
    project: str,
    display_name: str,
    location = "us-central1",
):
    aiplatform.init(project=project, location=location, credentials=credentials)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint


In [14]:
endpoint = create_endpoint_sample("vertex-ai-tuto-380714", "endpoint-iris-fromlocal-001")

Creating Endpoint
Create Endpoint backing LRO: projects/856301062069/locations/us-central1/endpoints/4932228842296180736/operations/2106969736892907520
Endpoint created. Resource name: projects/856301062069/locations/us-central1/endpoints/4932228842296180736
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/856301062069/locations/us-central1/endpoints/4932228842296180736')
endpoint-iris-fromlocal-001
projects/856301062069/locations/us-central1/endpoints/4932228842296180736


In [16]:
endpoint.resource_name.split('/')[-1]

'4932228842296180736'

# Deploy the model to the endpoint 

In [43]:
from typing import Optional, Dict

def deploy_model_with_dedicated_resources_sample(
    project,
    model_name: str,
    credentials,
    location = "us-central1",
    machine_type="n1-standard-4",
    endpoint: Optional[aiplatform.Endpoint] = None,
    deployed_model_display_name: Optional[str] = None,
    traffic_percentage: Optional[int] = 0,
    traffic_split: Optional[Dict[str, int]] = None,
    min_replica_count: int = 1,
    max_replica_count: int = 1,
    accelerator_type: Optional[str] = None,
    accelerator_count: Optional[int] = None,
#     explanation_metadata: Optional[explain.ExplanationMetadata] = None,
#     explanation_parameters: Optional[explain.ExplanationParameters] = None,
#     metadata: Optional[Sequence[Tuple[str, str]]] = (),
    sync: bool = True,
):
    """
    model_name: A fully-qualified model resource name or model ID.
          Example: "projects/123/locations/us-central1/models/456" or
          "456" when project and location are initialized or passed.
    """

    aiplatform.init(project=project, credentials=credentials)

    model = aiplatform.Model(model_name=model_name, credentials=credentials)

    # The explanation_metadata and explanation_parameters should only be
    # provided for a custom trained model and not an AutoML model.
    model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        traffic_percentage=traffic_percentage,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        explanation_metadata=explanation_metadata,
        explanation_parameters=explanation_parameters,
        metadata=metadata,
        sync=sync,
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model


In [48]:
# model = deploy_model_with_dedicated_resources_sample(
#     endpoint= "4932228842296180736",
#     project="vertex-ai-tuto-380714",
#     model_name="projects/vertex-ai-tuto/locations/us-central1/models/iris-model-fromlocal-01",
#     credentials=credentials
# )

In [54]:
def deploy_model_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    model_name: str,
    deployed_model_display_name: str,
    credentials: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    timeout: int = 7200,
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.EndpointServiceClient(client_options=client_options, credentials=credentials)
    deployed_model = {
        # format: 'projects/{project}/locations/{location}/models/{model}'
        "model": model_name,
        "display_name": deployed_model_display_name,
        # `dedicated_resources` must be used for non-AutoML models
        "dedicated_resources": {
            "min_replica_count": 1,
            "machine_spec": {
                "machine_type": "n1-standard-2",
                # Accelerators can be used only if the model specifies a GPU image.
                # 'accelerator_type': aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
                # 'accelerator_count': 1,
            },
        },
    }
    # key '0' assigns traffic for the newly deployed model
    # Traffic percentage values must add up to 100
    # Leave dictionary empty if endpoint should not accept any traffic
    traffic_split = {"0": 100}
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )
    print("Long running operation:", response.operation.name)
    deploy_model_response = response.result(timeout=timeout)
    print("deploy_model_response:", deploy_model_response)


In [55]:
PROJECT_ID = "vertex-ai-tuto-380714"
LOCATION = "us-central1"
ENDPOINT_ID = endpoint.resource_name.split('/')[-1]
MODEL_ID = "1851401246997479424"
MODEL_NAME = f'projects/{PROJECT_ID}/locations/{LOCATION}/models/{MODEL_ID}'

deploy_model_custom_trained_model_sample(
            project=PROJECT_ID,
            endpoint_id=ENDPOINT_ID,
            model_name=MODEL_ID,
            deployed_model_display_name = "iris-deployement-fromlocal-001",
            credentials= credentials
)

InvalidArgument: 400 List of found errors:	1.Field: deployed_model.model; Message: Invalid Model resource name.	 [field_violations {
  field: "deployed_model.model"
  description: "Invalid Model resource name."
}
]

# Get predictions

In [39]:
import os 

PROJECT_ID = os.getenv("BUILD_SPECIFIC_GCLOUD_PROJECT")

In [40]:
PROJECT_ID